In [16]:
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as fft
import IPython
from ipywidgets import interact
from scipy import signal
from scipy.signal import butter
import matplotlib as mpl

In [17]:
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'Serif', size = 16)

In [ ]:
fs, x = wavfile.read('pticice_avion.wav')
IPython.display.display(IPython.display.Audio(x, rate=fs))
N = len(x)
f = np.arange(N) * fs / N
X = fft.fft(x)

In [19]:
@interact(gain_bass=(0, 3, 0.001), gain_tremble = (0, 3, 0.001))
def amplitude(gain_bass, gain_tremble):
    fg=1e3
    Y = X.copy()
    Y[f>fg] *= gain_tremble
    Y[f<=fg] *= gain_bass

    for k in range(1, N//2):
        Y[N - k] = np.conj(Y[k])
        
    plt.title('Bass/tremble equalizer:')
    plt.plot(f[:N//2], abs(X)[:N//2], label='$Input$')
    plt.plot(f[:N//2], abs(Y)[:N//2], label='$Output$')
    plt.grid()
    plt.ylabel('|Y[k]|'), plt.xlabel('f')
    plt.legend()
    plt.show()
    y = np.real(fft.ifft(Y))
    IPython.display.display(IPython.display.Audio(y, rate=fs))

interactive(children=(FloatSlider(value=1.0, description='gain_bass', max=3.0, step=0.001), FloatSlider(value=…

In [20]:
@interact(freq_lowcut=(1, 22049), freq_highcut=(1, 22049), gain=(0, 3, 0.001))
def gen_signal(freq_lowcut, freq_highcut, gain):
    if freq_lowcut > freq_highcut:
        freq_highcut, freq_lowcut= freq_lowcut, freq_highcut
        
    filt = signal.butter(5, (freq_lowcut, freq_highcut), 'bandpass', fs=44100, output='sos')
    Y = signal.sosfilt(filt, gain*x)
    plt.title('Filter/gain applied signal:')
    plt.figure(1,figsize=(8, 5), dpi=70) 
    plt.plot(f[:N//2], abs(X)[:N//2], label='$Input$')
    plt.plot(f[:N//2], abs(fft.fft(Y))[:N//2], label='$Output$')
    plt.grid()
    plt.legend()
    plt.ylabel('|Y[k]|'), plt.xlabel('f')
    plt.show()
    
    filt1 = signal.butter(5, (1, freq_lowcut), 'bandpass', fs=44100, output='sos')
    Y1 = signal.sosfilt(filt1, x)
    plt.subplot(2,1,1)
    plt.plot(f[:N//2], abs(X)[:N//2])
    plt.plot(f[:N//2], abs(fft.fft(Y1))[:N//2])
    plt.grid()
    plt.ylabel('|Y1[k]|'), plt.xlabel('f')
    plt.show()
    
    filt2 = signal.butter(5, (freq_highcut, 22049), 'bandpass', fs=44100, output='sos')
    Y2 = signal.sosfilt(filt2, x)
    plt.subplot(2,1,2)
    plt.plot(f[:N//2], abs(X)[:N//2])
    plt.plot(f[:N//2], abs(fft.fft(Y2))[:N//2])
    plt.grid()
    plt.ylabel('|Y2[k]|'), plt.xlabel('f')
    plt.show()
    
    Z=Y+Y1+Y2
    plt.title('Final signal:')
    plt.figure(1,figsize=(8, 5), dpi=70)
    plt.plot(f[:N//2], abs(X)[:N//2], label='$Input$')
    plt.plot(f[:N//2], abs(fft.fft(Z))[:N//2], label='$Input$')
    plt.grid()
    plt.ylabel('|Z[k]|'), plt.xlabel('f')
    plt.legend()
    plt.show()
    IPython.display.display(IPython.display.Audio(Z, rate=fs))

interactive(children=(IntSlider(value=11025, description='freq_lowcut', max=22049, min=1), IntSlider(value=110…